In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb

def extract_features(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not read image {image_path}")
            return None

        # Convert to HSV color space
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # Calculate 3D histogram for HSV channels
        hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        return cv2.normalize(hist, hist).flatten()
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None

def prepare_dataset(jaundiced_dir, normal_dir):
    features = []
    labels = []

    # Load jaundiced images (class 1)
    if os.path.exists(jaundiced_dir):
        for img_file in os.listdir(jaundiced_dir):
            img_path = os.path.join(jaundiced_dir, img_file)
            feat = extract_features(img_path)
            if feat is not None:
                features.append(feat)
                labels.append(1)
    else:
        print(f"Warning: Jaundiced directory not found - {jaundiced_dir}")

    # Load normal images (class 0)
    if os.path.exists(normal_dir):
        for img_file in os.listdir(normal_dir):
            img_path = os.path.join(normal_dir, img_file)
            feat = extract_features(img_path)
            if feat is not None:
                features.append(feat)
                labels.append(0)
    else:
        print(f"Warning: Normal directory not found - {normal_dir}")

    if not features:
        raise ValueError("No valid images found in directories")

    return pd.DataFrame(features), pd.Series(labels)

def train_model(X, y):

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=150,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric='logloss'
    )

    model.fit(X_train, y_train)

    # Evaluate model
    if len(X) == 0 or len(y) == 0:
        raise ValueError("No training data available")


    # Evaluate model
    y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Print metrics
    print(f"\nModel Evaluation Metrics:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")

    return model


def predict_jaundice(model, image_path):
    features = extract_features(image_path)
    if features is not None:
        proba = model.predict_proba([features])[0]
        return {
            'prediction': 'Jaundiced' if np.argmax(proba) == 1 else 'Non-Jaundiced',
            'confidence': float(np.max(proba)),
            'jaundice_prob': float(proba[1])
        }
    return {'error': 'Could not process image'}

if __name__ == "__main__":
    # Configuration
    JAUNDICED_DIR = '/content/drive/MyDrive/Major Project Code/jaundiced_dir'  # Directory containing jaundiced cases
    NORMAL_DIR = '/content/drive/MyDrive/Major Project Code/normal_dir'        # Directory containing normal cases

    try:
        # Prepare dataset
        print("Loading and processing images...")
        X, y = prepare_dataset(JAUNDICED_DIR, NORMAL_DIR)

        # Train model
        print("\nTraining classification model...")
        model = train_model(X, y)

        # Example prediction
        test_image = os.path.join(JAUNDICED_DIR, '/content/drive/MyDrive/Major Project Code/normal_dir/normal (1016).jpg')  # Replace with actual image
        print("\nSample prediction:")
        print(predict_jaundice(model, test_image))

    except Exception as e:
        print(f"\nError: {str(e)}")
        print("\nTROUBLESHOOTING:")
        print(f"1. Ensure directories exist: '{JAUNDICED_DIR}' and '{NORMAL_DIR}'")
        print("2. Verify directories contain valid image files (JPEG/PNG)")
        print("3. Check image file permissions")








Loading and processing images...

Training classification model...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:52:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model Evaluation Metrics:
Accuracy:  0.8882
Precision: 0.8333
Recall:    0.6757
F1-Score:  0.7463

Sample prediction:
{'prediction': 'Non-Jaundiced', 'confidence': 0.996609091758728, 'jaundice_prob': 0.0033909238409250975}
